In [1]:
# Data Wrangling
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import seaborn as sns
%matplotlib inline

# EDA
import klib

# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.base import ClassifierMixin
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# Evaluation
from sklearn.model_selection import cross_val_score

# Utility
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
from sklearn.feature_selection import RFE

import gc
import random
import re
from typing import List ,Dict, Tuple

from sklearn.metrics import *
from sklearn.model_selection import KFold, StratifiedKFold

# 한글 폰트 설정
from statsmodels import robust
from matplotlib import font_manager, rc
%matplotlib inline

import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

In [2]:
DATA_PATH = "data/"
SUBMIT_PATH = "submission/"
SEED = 42

In [3]:
train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')

d_code = pd.read_csv(f'{DATA_PATH}속성_D_코드.csv').iloc[:, :-1]
h_code = pd.read_csv(f'{DATA_PATH}속성_H_코드.csv')
l_code = pd.read_csv(f'{DATA_PATH}속성_L_코드.csv')

train.shape , test.shape

((501951, 35), (46404, 34))

## preprocessing & engineering

### 데이터 - 속성코드 매칭

In [4]:
def add_code(df, d_code, h_code, l_code):
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_u'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 상위코드'])
    df['person_prefer_h_2_u'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 상위코드'])
    df['person_prefer_h_3_u'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 상위코드'])
    df['contents_attribute_h_u'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 상위코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    return df

In [5]:
code_d = pd.read_csv(f'{DATA_PATH}속성_D_코드.csv', index_col=0).iloc[:, :4].T.to_dict()
code_h = pd.read_csv(f'{DATA_PATH}속성_H_코드.csv', index_col=0).T.to_dict()
code_l = pd.read_csv(f'{DATA_PATH}속성_L_코드.csv', index_col=0).T.to_dict()

In [6]:
train = add_code(train, code_d, code_h, code_l)
test = add_code(test, code_d, code_h, code_l)

In [7]:
train.shape, test.shape

((501951, 59), (46404, 58))

### contents_open_dt 관련 시계열 피처
1. 연월일시간 중에 어떤 요소가 중요한가

In [8]:
train['contents_open_dt'] = pd.to_datetime(train['contents_open_dt'])
test['contents_open_dt'] = pd.to_datetime(test['contents_open_dt'])

In [9]:
#train['contents_open_year'] = train['contents_open_dt'].dt.year
train['contents_open_month'] = train['contents_open_dt'].dt.month
train['contents_open_day'] = train['contents_open_dt'].dt.day
train['contents_open_dow'] = train['contents_open_dt'].dt.dayofweek
train['contents_open_hour'] = train['contents_open_dt'].dt.hour

#test['contents_open_year'] = test['contents_open_dt'].dt.year
test['contents_open_month'] = test['contents_open_dt'].dt.month
test['contents_open_day'] = test['contents_open_dt'].dt.day
test['contents_open_dow'] = test['contents_open_dt'].dt.dayofweek
test['contents_open_hour'] = test['contents_open_dt'].dt.hour

In [10]:
train['weekend'] = train['contents_open_dow'].apply(lambda x : 1 if x > 4 else 0)
test['weekend'] = test['contents_open_dow'].apply(lambda x : 1 if x > 4 else 0)

### 회원속성

In [11]:
train2 = train.sort_values(by='contents_open_dt').reset_index(drop=True)
test2 = test.sort_values(by='contents_open_dt').reset_index(drop=True)

In [12]:
train2['누적_컨텐츠접촉'] = train2.groupby('person_rn')['person_rn'].cumcount() + 1
test2['누적_컨텐츠접촉'] = test2.groupby('person_rn')['person_rn'].cumcount() + 1

In [13]:
train['누적_컨텐츠접촉'] = train2.sort_values(by='id').reset_index()['누적_컨텐츠접촉']
test['누적_컨텐츠접촉'] = test2.sort_values(by='id').reset_index()['누적_컨텐츠접촉']

### 회원속성 - 컨텐츠 속성 일치 여부 

In [14]:
d_code.columns = ["attribute_d", "attribute_d_d", "attribute_d_s", "attribute_d_m", "attribute_d_l"]
h_code.columns = ["attribute_h", "attribute_h_p"]
l_code.columns = ["attribute_l", "attribute_l_d", "attribute_l_s", "attribute_l_m", "attribute_l_l"]

In [15]:
def merge_codes(df:pd.DataFrame,df_code:pd.DataFrame,col:str)->pd.DataFrame:
    df = df.copy()
    df_code = df_code.copy()
    df_code = df_code.add_prefix(f"{col}_")
    df_code.columns.values[0] = col
    return pd.merge(df,df_code,how="left", on=col)

In [16]:
def preprocess_data(
                    df:pd.DataFrame,is_train:bool = True, cols_merge:List[Tuple[str,pd.DataFrame]] = [], cols_equi:List[Tuple[str,str]]= [] ,
                    cols_drop:List[str] = ["id", "person_prefer_f", "person_prefer_g", "contents_open_dt"]
                    )->Tuple[pd.DataFrame,np.ndarray]:
    df = df.copy()

    y_data = None
    if is_train:
        y_data = df["target"].to_numpy()
        df = df.drop(columns="target")

    for col, df_code in cols_merge:
        df = merge_codes(df, df_code, col)

    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] = df[cols].astype(int)

    for col1, col2 in cols_equi:
        df[f"{col1}_{col2}"] = (df[col1] == df[col2]).astype(int)

    df = df.drop(columns=cols_drop)
    return (df, y_data)

In [17]:
# 소분류 중분류 대분류 속성코드 merge 컬럼명 및 데이터 프레임 리스트
cols_merge = [
              ("person_prefer_d_1" , d_code),
              ("person_prefer_d_2" , d_code),
              ("person_prefer_d_3" , d_code),
              ("contents_attribute_d" , d_code),
              ("person_prefer_h_1" , h_code),
              ("person_prefer_h_2" , h_code),
              ("person_prefer_h_3" , h_code),
              ("contents_attribute_h" , h_code),
              ("contents_attribute_l" , l_code),
]

# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [

    ("contents_attribute_c", "person_prefer_c"),
    ("contents_attribute_e", "person_prefer_e"),

    ("person_prefer_d_2_attribute_d_s", "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m", "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_l", "contents_attribute_d_attribute_d_l"),
    ("person_prefer_d_3_attribute_d_s", "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m", "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_l", "contents_attribute_d_attribute_d_l"),

    ("person_prefer_h_1_attribute_h_p", "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_2_attribute_h_p", "contents_attribute_h_attribute_h_p"),
    ("person_prefer_h_3_attribute_h_p", "contents_attribute_h_attribute_h_p"),

]

# 학습에 필요없는 컬럼 리스트
cols_drop = ["id", "person_prefer_f", "person_prefer_g", "contents_open_dt", "contents_rn"]

In [18]:
x_train, y_train = preprocess_data(train, cols_merge=cols_merge, cols_equi=cols_equi, cols_drop=cols_drop)
x_test, _ = preprocess_data(test, is_train=False, cols_merge=cols_merge, cols_equi=cols_equi, cols_drop=cols_drop)
x_train.shape, y_train.shape, x_test.shape

((501951, 94), (501951,), (46404, 94))

### 군집분석

In [19]:
from sklearn.cluster import KMeans

class KMeansFeaturizer:
    """ 숫자 데이터를 k-평균 클러스터 멤버십으로 변환.

    이 변환기는 입력 데이터에 k-평균을 수행해 각 데이터 포인트를 가장 가까운 클러스터의 id로 변환한다.
    만약 목표 변수가 주어지면 유사한 데이터 포인트와 함께 grouping되고,
    분류 경계에 따르는 클러스터를 생성하기 위해 스케일링되고, k-평균 입력에 포함된다.
    """

    def __init__(self, k = 100, target_scale = 5.0, random_state = None):
        self.k = k
        self.target_scale = target_scale
        self.random_state = random_state

    def fit(self, X, y = None):
        """ 입력 데이터에 k-평균을 수행하고 중심점을 찾는다.
        """
        if y is None: # 목표 변수가 없으면 단순한 k-평균 수행
            km_model = KMeans(n_clusters = self.k, n_init = 20, random_state = self.random_state)
            km_model.fit(X)
            
            self.inertia_ = km_model.inertia_
            self.km_model = km_model
            self.cluster_centers_ = km_model.cluster_centers_
            return self

        # 목표 변수가 있으면, 적절한 스케일링을 적용하고, 이를 k-평균에 대한 입력 데이터에 포함시킨다.
        data_with_target = np.hstack((X, y[:, np.newaxis] * self.target_scale))
        # 데이터와 타겟에 대해 사전 학습할 k-평균 모델 구축
        km_model_pretrain = KMeans(n_clusters = self.k, n_init = 20, random_state = self.random_state)
        km_model_pretrain.fit(data_with_target)

        # k평균을 두번째로 실행해 목표 변수 없이 원시 공간에서 클러스터를 얻는다. 사전 학습을 통해 얻은 중심점을 활용해 초기화한다.
        # 반복을 통해 클러스터 할당과 중심점 계산을 다시 수행한다.

        km_model = KMeans(n_clusters = self.k, init = km_model_pretrain.cluster_centers_[:,:data_with_target.shape[1]-1], n_init = 1, max_iter = 1)

        km_model.fit(X)
        
        self.inertia_ = km_model.inertia_
        self.km_model = km_model
        self.cluster_centers_ = km_model.cluster_centers_
        return self

    def transform(self, X, y = None):
        """ 각 입력 데이터 포인트에 대해 가장 가까운 클러스터 ID 산출
        """
        clusters = self.km_model.predict(X)
        return clusters[:, np.newaxis]

    def fit_transform(self, X, y = None):
        self.fit(X, y)
        return self.transform(X, y)

In [20]:
km = KMeansFeaturizer(k=100, random_state=42)

In [21]:
ktr = km.fit_transform(x_train, train["target"])
kte = km.transform(x_test)

In [22]:
x_train['km'] = ktr.astype(str)
x_test['km'] = kte.astype(str)

In [30]:
x_train['km'].value_counts()

12    6950
36    6813
70    6742
7     6594
74    6536
      ... 
4     3716
97    3578
22    3569
85    3505
47    3390
Name: km, Length: 100, dtype: int64

In [31]:
x_test['km'].value_counts()

19    1488
98    1395
27    1046
49    1039
65    1013
      ... 
96      32
44      26
89      24
38      20
77      15
Name: km, Length: 100, dtype: int64

### 범주형 칼럼 리스트

In [32]:
cat_features = x_train.columns[x_train.nunique() > 2].tolist()

In [33]:
cat_features = set(cat_features) - set(['누적_컨텐츠접촉'])

In [8]:
x_train

NameError: name 'x_train' is not defined

### 학습 파라미터

In [35]:
is_holdout = False
n_splits = 5
iterations = 3000
patience = 50

cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)

### 학습

In [36]:
scores = []
models = []


models = []
for tri, vai in cv.split(x_train):
    print("="*50)
    preds = []

    model = CatBoostClassifier(iterations=iterations, 
                               random_state=SEED,
                               #task_type="GPU",
                               eval_metric="F1",
                               cat_features=cat_features,
                               one_hot_max_size=4)
    model.fit(x_train.iloc[tri], y_train[tri], 
            eval_set=[(x_train.iloc[vai], y_train[vai])], 
            early_stopping_rounds=patience ,
            verbose = 100
        )
    
    models.append(model)
    scores.append(model.get_best_score()["validation"]["F1"])
    if is_holdout:
        break    

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.086395
0:	learn: 0.6271045	test: 0.6342993	best: 0.6342993 (0)	total: 3.51s	remaining: 2h 55m 19s
100:	learn: 0.6646329	test: 0.6842006	best: 0.6845411 (96)	total: 8m 22s	remaining: 4h 10s
200:	learn: 0.6716887	test: 0.6860758	best: 0.6860758 (200)	total: 16m 49s	remaining: 3h 54m 20s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6860763862
bestIteration = 201

Shrink model to first 202 iterations.
Learning rate set to 0.086395
0:	learn: 0.6249856	test: 0.6300820	best: 0.6300820 (0)	total: 3.09s	remaining: 2h 34m 26s
100:	learn: 0.6646489	test: 0.6856481	best: 0.6865826 (96)	total: 11m 49s	remaining: 5h 42m 45s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.6875319318
bestIteration = 146

Shrink model to first 147 iterations.
Learning rate set to 0.086395
0:	learn: 0.6228982	test: 0.6300020	best: 0.6300020 (0)	total: 4.08s	remaining: 3h 24m 8s
100:	learn: 0.6640610	test: 0.6799150	best: 0.6813235 (96)	total: 9m 32s	remai

### cv 결과 확인

In [37]:
print(scores)
print(np.mean(scores))

[0.6860763862259377, 0.6875319318031298, 0.6813235027612716, 0.6925713858713419, 0.6781437972583108]
0.6851294007839984


### threshold 정의

In [4]:
threshold = 0.4

### threshold값 변경에 따른 검증점수 확인 및 추론

In [39]:
pred_list = []
scores = []
for i,(tri, vai) in enumerate( cv.split(x_train) ):
    pred = models[i].predict_proba(x_train.iloc[vai])[:, 1]
    pred = np.where(pred >= threshold , 1, 0)
    score = f1_score(y_train[vai],pred)
    scores.append(score)
    pred = models[i].predict_proba(x_test)[:, 1]
    pred_list.append(pred)
print(scores)
print(np.mean(scores))

[0.7061393037465403, 0.7076019314239894, 0.7024781124795965, 0.7090073896544837, 0.7004102555281287]
0.7051273985665477


### 산술평균 앙상블

In [40]:
pred = np.mean(pred_list, axis=0)
pred = np.where(pred >= threshold, 1, 0)

### submission

In [41]:
sample_submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
sample_submission['target'] = pred
sample_submission

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
46399,46399,1
46400,46400,1
46401,46401,1
46402,46402,1


In [42]:
sample_submission['target'].value_counts()

1    29254
0    17150
Name: target, dtype: int64

In [43]:
sample_submission.to_csv(f"{SUBMIT_PATH}jp_1223_5.csv", index=False)